In [1]:
%run src_runs/autorun/daily_update.py

TypeError: function() argument 'code' must be code, not str

In [2]:
import pandas as pd

df = pd.read_feather('./data/DataBase/DB_models/risk_exp/2024/risk_exp.20240102.feather')
df

,secid,estuniv,weight,market,petro,coal,nonferrous,utility,public,steel,...,size,beta,momentum,residual_volatility,non_linear_size,book_to_price,liquidity,earnings_yield,growth,leverage
0,1,1.0,1787.250854,1,0,0,0,0,0,0,...,1.110746,0.188975,-1.303870,-0.579935,-0.614874,2.123760,0.013252,1.856212,-0.020907,1.463905
1,2,1.0,986.241699,1,0,0,0,0,0,0,...,1.110746,0.766382,-1.291235,-0.134110,-0.614874,2.123760,0.187936,1.398134,-0.458002,1.239772
2,4,NaN,NaN,1,0,0,0,0,0,0,...,-2.934183,0.257048,1.648986,1.584003,-2.097585,-1.349531,1.655931,-3.009397,-2.216894,-1.243104
3,5,0.0,0.000000,1,0,0,0,0,1,0,...,-3.558902,-1.279942,-0.622580,0.708833,-2.550303,1.589081,-1.319573,-3.038959,-1.684412,-0.923338
4,6,1.0,60.344437,1,0,0,0,0,0,0,...,-1.832967,-0.469602,0.160499,1.007087,0.892483,2.123760,0.173596,0.042151,-0.766682,1.173919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,688799,1.0,27.085997,1,0,0,0,0,0,0,...,-2.144708,0.453999,1.192576,-0.011164,0.417225,-0.698647,0.460217,-0.116496,0.244113,-1.542007
5092,688800,1.0,42.635628,1,0,0,0,0,0,0,...,-1.781119,0.598133,-1.623481,1.111750,0.947838,-0.894435,-0.218326,-0.769623,1.352287,-1.122012
5093,688819,1.0,36.782902,1,0,0,0,0,0,0,...,-0.231432,0.189262,-1.033293,-0.593890,0.519714,-0.199649,0.094145,0.960113,-0.436809,-0.241816
5094,688981,1.0,1045.223389,1,0,0,0,0,0,0,...,1.110746,-0.090532,0.626771,0.438562,-0.614874,-0.829069,0.534798,-0.807107,1.032421,0.418790


In [1]:
from src.api import DataAPI , ModelPredictor , ModelHiddenExtractor

try:
    DataAPI.update()
except Exception as e:
    print(f'DataAPI went wrong: {e}')

print('prepare_predict_data')
DataAPI.prepare_predict_data()

ModelHiddenExtractor.update_hidden('gru_day' , model_submodels=['best'])
ModelPredictor.update_factors()

at db_dates , benchmark will be deprecated soon, please update your code
at db_dates , benchmark will be deprecated soon, please update your code
at db_dates , benchmark will be deprecated soon, please update your code
at db_dates , benchmark will be deprecated soon, please update your code
Update Files
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db_dates , trade will be deprecated soon, please update your code
at db

100%|██████████| 316/316 [00:00<00:00, 3163.22it/s]


--------------------------------------------------------------------------------


  0%|          | 0/107 [00:00<?, ?it/s]

Finish model [gru_day] predicting!


  0%|          | 0/107 [00:00<?, ?it/s]

Finish model [gruRTN_day] predicting!


  0%|          | 0/107 [00:00<?, ?it/s]

Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


ModelPredictor(reg_model=RegisteredModel(name=gru_avg,type=swalast,num=all,alias=gru_day_V1))

In [1]:
from src.api import DataAPI , ModelPredictor , ModelHiddenExtractor

DataAPI.update()

Update Files
Wed Nov 13 22:45:20 2024 : download since!
Connection and Factor preparation finished!
Wed Nov 13 22:45:20 2024 : All Updates Done! Cost 36.64 Secs
--------------------------------------------------------------------------------


c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  22 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


--------------------------------------------------------------------------------
models , tushare_cne5_exp , 20241111 saved
models , tushare_cne5_coef , 20241111 saved
models , tushare_cne5_res , 20241111 saved
models , tushare_cne5_exp , 20241112 saved
models , tushare_cne5_coef , 20241112 saved
models , tushare_cne5_res , 20241112 saved
models , tushare_cne5_exp , 20241113 saved
models , tushare_cne5_coef , 20241113 saved
models , tushare_cne5_res , 20241113 saved
models , tushare_cne5_cov , 20241111 saved
models , tushare_cne5_spec , 20241111 saved
models , tushare_cne5_cov , 20241112 saved
models , tushare_cne5_spec , 20241112 saved
models , tushare_cne5_cov , 20241113 saved
models , tushare_cne5_spec , 20241113 saved
--------------------------------------------------------------------------------
labels_ts , ret5 , 20240102 saved
labels_ts , ret5 , 20240103 saved
labels_ts , ret5 , 20240104 saved
labels_ts , ret5 , 20240105 saved
labels_ts , ret5 , 20240108 saved
labels_ts , ret5 

In [1]:
from src.api import DataAPI , ModelPredictor , ModelHiddenExtractor
from pymysql.err import OperationalError

print('prepare_predict_data')
DataAPI.prepare_predict_data()


ModelHiddenExtractor.update_hidden('gru_day' , model_submodels=['best'])
ModelPredictor.update_factors()

prepare_predict_data
predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 2.79 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 2.27 secs
 --> labels blocks merging (2)...... finished! Cost 0.39 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 5.45 secs
y blocks loading finished! Cost 11.57 secs
y blocks process...... finished! Cost 11.99 secs
y blocks masking...... finished! Cost 0.18 secs
y blocks saving ...... finished! Cost 0.23 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 24.13 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 2.46 secs
day blocks loading finished! Cost 2.50 secs
day blocks process...... finished! Cost 0.20 secs
day blocks masking...

100%|██████████| 316/316 [00:00<00:00, 1676.43it/s]


--------------------------------------------------------------------------------


  0%|          | 0/96 [00:00<?, ?it/s]

Finish model [gru_day] predicting!


  0%|          | 0/96 [00:00<?, ?it/s]

Finish model [gruRTN_day] predicting!


  0%|          | 0/96 [00:00<?, ?it/s]

Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


ModelPredictor(reg_model=RegisteredModel(name=gru_avg,type=swalast,num=all,alias=gru_day_V1))

In [1]:
from src.api import DataAPI , ModelPredictor

DataAPI.update()
DataAPI.prepare_predict_data()
ModelPredictor.update_factors()

Update Files
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\calendar.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\description.feather Done! Cost 0.03 Secs
Thu Sep 19 00:49:45 2024 : Updated ~ DB_information\st.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:48 2024 : Updated ~ DB_information\industry.feather Done! Cost 2.55 Secs
Thu Sep 19 00:49:53 2024 : Updated ~ DB_information\concepts.feather Done! Cost 5.54 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\longcl_exp\2024\longcl_exp.20240913.feather Done! Cost 0.10 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_exp\2024\risk_exp.20240918.feather Done! Cost 0.05 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_cov\2024\risk_cov.20240918.feather Done! Cost 0.02 Secs
Thu Sep 19 00:49:55 2024 : Updated ~ DB_models\risk_spec\2024\risk_spec.20240918.feather Done! Cost 0.01 Secs
Thu Sep 19 00:49:57 2024 : Updated ~ DB_trade\day\2024\day.20240918.feather Done! Cost 0.05 Secs
Thu S

Predictor(reg_model=RegModel(name=gru_avg,type=swabest,num=all,alias=gru_day_V1))

In [8]:
from src.data.tushare.download.info import Calendar , pro
renamer = {'cal_date' : 'calendar' ,'is_open'  : 'trade'}
fields = None
df = pro.trade_cal(exchange='SSE').rename(columns=renamer)
#df = df.sort_values('calendar').reset_index(drop = True)
df

""


In [10]:
# DataAPI.prepare_predict_data()
ModelPredictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [2]:
from src.api import DataAPI , ModelPredictor
# DataAPI.prepare_predict_data()
md = ModelPredictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------


In [ ]:
from src.api import ModelTestor
model = ModelTestor('ts_mixer')
model.try_forward()
model.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
